<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.3:
# *Database Programming with Python and SQLite*

In the first part of the lab you will use install and use SQLite, a database program with which we can create and query small databases. The Python library `sqlite3` will be used to connect the database to this Jupyter notebook.

In the second part of the lab, two dataframes will be saved as tables in the SQLite database. Then a number of questions about the data will be asked to practise SQL syntax.

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and database maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


In [114]:
$ sqlite3 ex1

SyntaxError: invalid syntax (1410101139.py, line 1)

Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands".

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [4]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [7]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [6]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

OperationalError: table houses already exists

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [8]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key.

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [12]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
#
c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

Select all rows from houses

In [27]:
# A
query = 'Select * from houses'
c.execute(query)
c.fetchall()
#pd.DataFrame(c.fetchall())

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more.

In [18]:
from numpy import genfromtxt
housing_data = r'C:\Users\pabarca\OneDrive - GRUPO GRANSOLAR\Desktop\IOD - Python\DATA\housing-data.csv'
# import into nparray of ints, then convert to list of lists:
data = genfromtxt(housing_data, dtype = 'i8',
                    delimiter = ',', skip_header = 1).tolist()

data

[[2104, 3, 70, 399900],
 [1600, 3, 28, 329900],
 [2400, 3, 44, 369000],
 [1416, 2, 49, 232000],
 [3000, 4, 75, 539900],
 [1985, 4, 61, 299900],
 [1534, 3, 12, 314900],
 [1427, 3, 57, 198999],
 [1380, 3, 14, 212000],
 [1494, 3, 15, 242500],
 [1940, 4, 7, 239999],
 [2000, 3, 27, 347000],
 [1890, 3, 45, 329999],
 [4478, 5, 49, 699900],
 [1268, 3, 58, 259900],
 [2300, 4, 77, 449900],
 [1320, 2, 62, 299900],
 [1236, 3, 78, 199900],
 [2609, 4, 5, 499998],
 [3031, 4, 21, 599000],
 [1767, 3, 44, 252900],
 [1888, 2, 79, 255000],
 [1604, 3, 13, 242900],
 [1962, 4, 53, 259900],
 [3890, 3, 36, 573900],
 [1100, 3, 60, 249900],
 [1458, 3, 29, 464500],
 [2526, 3, 13, 469000],
 [2200, 3, 28, 475000],
 [2637, 3, 25, 299900],
 [1839, 2, 40, 349900],
 [1000, 1, 5, 169900],
 [2040, 4, 75, 314900],
 [3137, 3, 67, 579900],
 [1811, 4, 24, 285900],
 [1437, 3, 50, 249900],
 [1239, 3, 22, 229900],
 [2132, 4, 28, 345000],
 [4215, 4, 66, 549000],
 [2162, 4, 43, 287000],
 [1664, 2, 40, 368500],
 [2238, 3, 37, 3299

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [22]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

In [24]:
data

[[None, 2104, 3, 70, 399900],
 [None, 1600, 3, 28, 329900],
 [None, 2400, 3, 44, 369000],
 [None, 1416, 2, 49, 232000],
 [None, 3000, 4, 75, 539900],
 [None, 1985, 4, 61, 299900],
 [None, 1534, 3, 12, 314900],
 [None, 1427, 3, 57, 198999],
 [None, 1380, 3, 14, 212000],
 [None, 1494, 3, 15, 242500],
 [None, 1940, 4, 7, 239999],
 [None, 2000, 3, 27, 347000],
 [None, 1890, 3, 45, 329999],
 [None, 4478, 5, 49, 699900],
 [None, 1268, 3, 58, 259900],
 [None, 2300, 4, 77, 449900],
 [None, 1320, 2, 62, 299900],
 [None, 1236, 3, 78, 199900],
 [None, 2609, 4, 5, 499998],
 [None, 3031, 4, 21, 599000],
 [None, 1767, 3, 44, 252900],
 [None, 1888, 2, 79, 255000],
 [None, 1604, 3, 13, 242900],
 [None, 1962, 4, 53, 259900],
 [None, 3890, 3, 36, 573900],
 [None, 1100, 3, 60, 249900],
 [None, 1458, 3, 29, 464500],
 [None, 2526, 3, 13, 469000],
 [None, 2200, 3, 28, 475000],
 [None, 2637, 3, 25, 299900],
 [None, 1839, 2, 40, 349900],
 [None, 1000, 1, 5, 169900],
 [None, 2040, 4, 75, 314900],
 [None, 3137,

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [26]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [28]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In [30]:
data

[[None, 2104, 3, 70, 399900],
 [None, 1600, 3, 28, 329900],
 [None, 2400, 3, 44, 369000],
 [None, 1416, 2, 49, 232000],
 [None, 3000, 4, 75, 539900],
 [None, 1985, 4, 61, 299900],
 [None, 1534, 3, 12, 314900],
 [None, 1427, 3, 57, 198999],
 [None, 1380, 3, 14, 212000],
 [None, 1494, 3, 15, 242500],
 [None, 1940, 4, 7, 239999],
 [None, 2000, 3, 27, 347000],
 [None, 1890, 3, 45, 329999],
 [None, 4478, 5, 49, 699900],
 [None, 1268, 3, 58, 259900],
 [None, 2300, 4, 77, 449900],
 [None, 1320, 2, 62, 299900],
 [None, 1236, 3, 78, 199900],
 [None, 2609, 4, 5, 499998],
 [None, 3031, 4, 21, 599000],
 [None, 1767, 3, 44, 252900],
 [None, 1888, 2, 79, 255000],
 [None, 1604, 3, 13, 242900],
 [None, 1962, 4, 53, 259900],
 [None, 3890, 3, 36, 573900],
 [None, 1100, 3, 60, 249900],
 [None, 1458, 3, 29, 464500],
 [None, 2526, 3, 13, 469000],
 [None, 2200, 3, 28, 475000],
 [None, 2637, 3, 25, 299900],
 [None, 1839, 2, 40, 349900],
 [None, 1000, 1, 5, 169900],
 [None, 2040, 4, 75, 314900],
 [None, 3137,

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [66]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

In [84]:
data1 = c.execute('SELECT * FROM houses')
conn.commit()
data1.fetchall()


[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000),
 (5, 2104, 3, 70, 399900),
 (6, 1600, 3, 28, 329900),
 (7, 2400, 3, 44, 369000),
 (8, 1416, 2, 49, 232000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (11, 1534, 3, 12, 314900),
 (12, 1427, 3, 57, 198999),
 (13, 1380, 3, 14, 212000),
 (14, 1494, 3, 15, 242500),
 (15, 1940, 4, 7, 239999),
 (16, 2000, 3, 27, 347000),
 (17, 1890, 3, 45, 329999),
 (18, 4478, 5, 49, 699900),
 (19, 1268, 3, 58, 259900),
 (20, 2300, 4, 77, 449900),
 (21, 1320, 2, 62, 299900),
 (22, 1236, 3, 78, 199900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (25, 1767, 3, 44, 252900),
 (26, 1888, 2, 79, 255000),
 (27, 1604, 3, 13, 242900),
 (28, 1962, 4, 53, 259900),
 (29, 3890, 3, 36, 573900),
 (30, 1100, 3, 60, 249900),
 (31, 1458, 3, 29, 464500),
 (32, 2526, 3, 13, 469000),
 (33, 2200, 3, 28, 475000),
 (34, 2637, 3, 25, 299900),
 (35, 1839, 2, 40, 349900),
 (36, 1000, 1, 5, 169900),
 (37

In [98]:
data1.description

(('field1', None, None, None, None, None, None),
 ('sqft', None, None, None, None, None, None),
 ('bdrms', None, None, None, None, None, None),
 ('age', None, None, None, None, None, None),
 ('price', None, None, None, None, None, None))

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [118]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

data1 = c.execute('SELECT * FROM houses')
conn.commit()
data1.fetchall()


[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000),
 (5, 2104, 3, 70, 399900),
 (6, 1600, 3, 28, 329900),
 (7, 2400, 3, 44, 369000),
 (8, 1416, 2, 49, 232000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (11, 1534, 3, 12, 314900),
 (12, 1427, 3, 57, 198999),
 (13, 1380, 3, 14, 212000),
 (14, 1494, 3, 15, 242500),
 (15, 1940, 4, 7, 239999),
 (16, 2000, 3, 27, 347000),
 (17, 1890, 3, 45, 329999),
 (18, 4478, 5, 49, 699900),
 (19, 1268, 3, 58, 259900),
 (20, 2300, 4, 77, 449900),
 (21, 1320, 2, 62, 299900),
 (22, 1236, 3, 78, 199900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (25, 1767, 3, 44, 252900),
 (26, 1888, 2, 79, 255000),
 (27, 1604, 3, 13, 242900),
 (28, 1962, 4, 53, 259900),
 (29, 3890, 3, 36, 573900),
 (30, 1100, 3, 60, 249900),
 (31, 1458, 3, 29, 464500),
 (32, 2526, 3, 13, 469000),
 (33, 2200, 3, 28, 475000),
 (34, 2637, 3, 25, 299900),
 (35, 1839, 2, 40, 349900),
 (36, 1000, 1, 5, 169900),
 (37

QUIZ: What would `DELETE FROM houses` do?  

In [ ]:
Deletes rows in column field1 that has a value of 52 or 53

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [120]:
# ANSWER

results = c.execute('SELECT * FROM houses WHERE bdrms = 4')
results.fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. The following query calculates the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [ ]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1814.1923076923076, 323465.3076923077),
 (4, 2400.4285714285716, 377449.78571428574),
 (5, 4478.0, 699900.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [124]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database.

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [2]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:


data = pd.read_csv(r'C:\Users\pabarca\OneDrive - GRUPO GRANSOLAR\Desktop\IOD - Python\DATA\housing-data.csv', low_memory = False)
data.head()

NameError: name 'pd' is not defined

We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [130]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

47

Run a query to get the average price of each house size from this table:

In [136]:
#ANSWER
c.execute('SELECT bdrms, AVG(price) FROM houses_pandas GROUP BY bdrms').fetchall()


[(1, 169900.0),
 (2, 280866.6666666667),
 (3, 326403.92),
 (4, 377449.78571428574),
 (5, 699900.0)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

In [ ]:
#Pandas is useful for cleaning data step by step, allowing you to easily inspect intermediate results. 
#Once the data is cleaned, it can be stored in a database.

#It is also well-suited for statistical analysis and can help prepare datasets for visualization.

<a name="guided-practice"></a>


# Reference: SQL Syntax


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate functions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [143]:
# Reading CSV to Dataframe
orders = pd.read_csv(r'C:\Users\pabarca\OneDrive - GRUPO GRANSOLAR\Desktop\IOD - Python\DATA\P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv(r'C:\Users\pabarca\OneDrive - GRUPO GRANSOLAR\Desktop\IOD - Python\DATA\P12-OrderBreakdown.csv', encoding = 'utf-8')

In [145]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [147]:
orders_break_down.head()

,Order ID,Product Name,Discount,Unit Price,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [149]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [151]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes

In [153]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [155]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
unit_price      float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [161]:
# Establishing Local DB connection
db_connection = sqlite3.connect('eshop.db.sqlite')

# Save these two dataframes as tables in sqlite
orders.to_sql(name = 'orders', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql(name = 'orders_break_down', con = db_connection, if_exists = 'replace', index = False)


8047

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [163]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT *
        FROM orders
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


#### 5. Find the number of orders for each customer

In [165]:
# Find the number of DISTINCT order ids for each customer name
sql_query = '''
    SELECT 
        customer_name,
        COUNT(DISTINCT order_id) AS Count
    FROM orders
    GROUP BY customer_name
'''

customers = sql.read_sql(sql_query, con = db_connection)
customers.head()

,customer_name,Count
0,Aaron Bergman,11
1,Aaron Hawkins,7
2,Aaron Smayling,12
3,Adam Bellavance,4
4,Adam Hart,8


#### 6. Find the number of customers for each country

In [171]:
# Find the number of customers for each country
sql_countries = '''
    SELECT 
        country,
        COUNT(DISTINCT order_id) AS Count
    FROM orders
    GROUP BY country
    ORDER BY Count DESC
'''

customers_country = sql.read_sql(sql_countries, con = db_connection)
customers_country.head()

,country,Count
0,France,991
1,Germany,806
2,United Kingdom,700
3,Italy,493
4,Spain,403


#### 7.A Select discount, unit price, quantity for each order from orders_break_down table

In [173]:
#  Select discount, unit price, quantity for each order from orders_break_down table
orders_break_down.head()

,order_id,product_name,discount,unit_price,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


In [183]:
sql_string = '''
    SELECT
        order_id,
        discount
        , unit_price
        , quantity
    FROM orders_break_down
'''
query = sql.read_sql(sql_string, con = db_connection)
query.head()


,order_id,discount,unit_price,quantity
0,IT-2011-3647632,0.5,45.0,3
1,ES-2011-4869686,0.0,854.0,7
2,ES-2011-4939443,0.0,140.0,3
3,IT-2011-2942451,0.5,27.0,2
4,IT-2011-2942451,0.5,17.0,2


#### 7.B Select discount, unit price, quantity, discounted price for each order from orders_break_down table

> discounted price = unit_price * quantity * (1 - discount)

In [187]:
# Select discount, unit price, quantity, discounted price for each order from orders_break_down table with discounted price in descending order
sql_string = '''
    SELECT
        order_id,
        discount
        , unit_price
        , quantity
        , (unit_price * quantity *(1- discount)) AS discounted_price
    FROM orders_break_down
    ORDER BY discounted_price DESC
'''
query = sql.read_sql(sql_string, con = db_connection)
query.head()


,order_id,discount,unit_price,quantity,discounted_price
0,ES-2011-3248922,0.0,6517.0,12,78204.0
1,ES-2014-3785216,0.1,5729.0,14,72185.4
2,ES-2012-2183106,0.1,4748.0,14,59824.8
3,ES-2014-2637201,0.0,4141.0,13,53833.0
4,ES-2013-2860574,0.0,5274.0,10,52740.0


#### 7.C Select all columns and also the discounted price from orders_break_down table where discounted price (defined above) is greater than 100

In [201]:
# Select all orders and also the discounted price from orders_break_down table where discounted price (defined above) is greater than 100,
# ordered by discounted price in ascending order

sql_string = '''
    SELECT
        order_id,
        discount
        , unit_price
        , quantity
        , (unit_price * quantity *(1- discount)) AS discounted_price
    FROM orders_break_down
    WHERE discounted_price > 100
    ORDER BY discounted_price DESC
   
'''
query = sql.read_sql(sql_string, con = db_connection)
query.head(100)


,order_id,discount,unit_price,quantity,discounted_price
0,ES-2011-3248922,0.0,6517.0,12,78204.0
1,ES-2014-3785216,0.1,5729.0,14,72185.4
2,ES-2012-2183106,0.1,4748.0,14,59824.8
3,ES-2014-2637201,0.0,4141.0,13,53833.0
4,ES-2013-2860574,0.0,5274.0,10,52740.0
...,...,...,...,...,...
95,ES-2014-1354154,0.0,1270.0,11,13970.0
96,ES-2012-4899743,0.1,2571.0,6,13883.4
97,IT-2014-5637241,0.1,1402.0,11,13879.8
98,ES-2011-5911716,0.0,1542.0,9,13878.0


#### 8. Select all order ids, customers and the product they have bought

In [207]:
# Select all order_ids, customer names and the products (product_name) they have bought
sql_string = '''
    SELECT
        orders.order_id,
        orders.customer_name,
        orders_break_down.product_name
    FROM orders
    JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
  
   
'''
query = sql.read_sql(sql_string, con = db_connection)
query.head(100)

,order_id,customer_name,product_name
0,IT-2011-3647632,Eugene Moren,"Enermax Note Cards, Premium"
1,ES-2011-4869686,Dorothy Dickinson,"Dania Corner Shelving, Traditional"
2,ES-2011-4939443,Arthur Prichep,"Binney & Smith Sketch Pad, Easy-Erase"
3,IT-2011-2942451,Grant Thornton,"Boston Markers, Easy-Erase"
4,IT-2011-2942451,Grant Thornton,"Eldon Folders, Single Width"
...,...,...,...
95,ES-2011-2722980,Hallie Redmond,"Ikea Corner Shelving, Metal"
96,ES-2011-2722980,Hallie Redmond,"Smead Folders, Wire Frame"
97,ES-2011-2722980,Hallie Redmond,"Stockwell Rubber Bands, 12 Pack"
98,ES-2011-4342226,Clay Ludtke,"Binney & Smith Pencil Sharpener, Fluorescent"


#### 9.A Find the number of 'Furniture' orders for each country

In [211]:
orders_break_down.head()

,order_id,product_name,discount,unit_price,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


In [209]:
orders.head()


,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [221]:
# Find the number of 'Furniture' orders for each country
sql_countries = '''
    SELECT 
        orders.country,
        COUNT(DISTINCT orders_break_down.category) AS Count
    FROM orders
    JOIN orders_break_down 
    ON orders.order_id = orders_break_down.order_id
    WHERE orders_break_down.category = 'Furniture'
    GROUP BY orders.country

'''

customers_country = sql.read_sql(sql_countries, con = db_connection)
customers_country.head()

,country,Count
0,Austria,1
1,Belgium,1
2,Denmark,1
3,Finland,1
4,France,1


In [227]:
sql_countries = '''
    SELECT 
        *
    FROM orders
    JOIN orders_break_down 
    ON orders.order_id = orders_break_down.order_id
    WHERE orders_break_down.category = 'Furniture'
    GROUP BY orders.country

'''

customers_country = sql.read_sql(sql_countries, con = db_connection)
customers_country.head()

,order_id,order_date,customer_name,country,order_id,product_name,discount,unit_price,quantity,category
0,ES-2011-4932761,2011-05-18,Sean O'Donnell,Austria,ES-2011-4932761,"Office Star Swivel Stool, Adjustable",0.0,1059.0,6,Furniture
1,ES-2011-2377967,2011-02-24,Julie Creighton,Belgium,ES-2011-2377967,"Bush Classic Bookcase, Mobile",0.0,2076.0,5,Furniture
2,ES-2011-1586186,2011-06-16,Vivek Grady,Denmark,ES-2011-1586186,"Office Star Bag Chairs, Set of Two",0.5,87.0,3,Furniture
3,ES-2011-3081212,2011-08-27,Victoria Wilson,Finland,ES-2011-3081212,"Novimex Rocking Chair, Set of Two",0.0,267.0,2,Furniture
4,ES-2011-5433855,2011-01-07,Dave Poirier,France,ES-2011-5433855,"Bush Floating Shelf Set, Pine",0.1,155.0,1,Furniture


#### 9.B Select 'Furniture' orders for the country Denmark

In [225]:
# Select all columns from the orders table which are 'Furniture' orders for the country Denmark
sql_countries = '''
    SELECT 
        *
    FROM orders
    JOIN orders_break_down 
    ON orders.order_id = orders_break_down.order_id
    WHERE orders_break_down.category = 'Furniture' AND orders.country = 'Denmark'
    GROUP BY orders.country

'''

customers_country = sql.read_sql(sql_countries, con = db_connection)
customers_country.head()

,order_id,order_date,customer_name,country,order_id,product_name,discount,unit_price,quantity,category
0,ES-2011-1586186,2011-06-16,Vivek Grady,Denmark,ES-2011-1586186,"Office Star Bag Chairs, Set of Two",0.5,87.0,3,Furniture


#### 10. Find the total sales with discount and without discount for each country

In [229]:
# Find the total sales with discount (discount > 0) and without discount (discount = 0) for each country
# total sales with discount = sum (unit price * quantity * (1 - discount)) for each country where we only sum over rows where discount > 0
# total sales without discount = sum (unit price * quantity) for each country where we only sum over rows where discount = 0
# hint: use case statements

sql_countries = '''
    SELECT 
        orders.country,
        SUM(CASE WHEN discount > 0 THEN unit_price * quantity *(1 - discount) ELSE 0 END) AS sales_with_discount ,
        SUM(CASE WHEN discount = 0 THEN unit_price * quantity ELSE 0 END) AS sales_without_discount
    FROM orders
    JOIN orders_break_down 
    ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country

'''

customers_country = sql.read_sql(sql_countries, con = db_connection)
customers_country.head()


,country,sales_with_discount,sales_without_discount
0,Austria,0.0,391694.0
1,Belgium,0.0,229829.0
2,Denmark,17117.8,0.0
3,Finland,0.0,82001.0
4,France,2063927.0,665914.0


#### 11.A Find the total quantity, total sales for each country

In [249]:
# Select sum of quantities, total sales (sum (unit_price * quantity * (1 - discount))) for each country
sql_countries = '''
    SELECT 
        orders.country,
        SUM(unit_price * quantity *(1 - discount)) AS total_sales_country,
        SUM(quantity) AS total_quantity
    FROM orders
    JOIN orders_break_down 
    ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country
   

'''

customers_country = sql.read_sql(sql_countries, con = db_connection)
customers_country.head()

,country,total_sales_country,total_quantity
0,Austria,391694.0,973
1,Belgium,229829.0,532
2,Denmark,17117.8,204
3,Finland,82001.0,201
4,France,2729841.0,7329


#### 11.B Based on total sales for each country, find the top 3 countries and their quantities

In [251]:
# Based on total sales for each country (sum (unit_price * quantity * (1 - discount))), find the top 3 countries and their quantities
sql_countries = '''
    SELECT 
        orders.country,
        SUM(unit_price * quantity *(1 - discount)) AS total_sales_country,
          SUM(quantity) AS total_quantity
    FROM orders
    JOIN orders_break_down 
    ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country
    ORDER BY total_sales_country DESC LIMIT 3

'''

customers_country = sql.read_sql(sql_countries, con = db_connection)
customers_country.head()

,country,total_sales_country,total_quantity
0,France,2729841.0,7329
1,Germany,2233458.8,6179
2,United Kingdom,2078715.7,4917


#### 11.C Based on total quantities for each country, find the bottom 3 countries, their total sales and total quantities

In [255]:
# Based on total quantities for each country, find the bottom 3 countries, their total sales (sum(unit_price * quantity * (1 - discount))) and total quantities
sql_countries = '''
    SELECT 
        orders.country,
        SUM(unit_price * quantity *(1 - discount)) AS total_sales_country,
        SUM(quantity) AS total_quantity
    FROM orders
    JOIN orders_break_down 
    ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country
    ORDER BY total_quantity ASC LIMIT 3

'''

customers_country = sql.read_sql(sql_countries, con = db_connection)
customers_country.head()

,country,total_sales_country,total_quantity
0,Finland,82001.0,201
1,Denmark,17117.8,204
2,Norway,99878.0,261


#### 12. Find average sales by category for the country 'France'

In [263]:
# Find average sales (i.e. avg( unit_price * quantity * (1 - discount) ) ) by category for the country 'France' in descending order
sql_countries = '''
    SELECT 
        orders.country,
        orders_break_down.category,
        AVG(unit_price * quantity * (1 - discount)) AS avg_sales
    FROM orders
    JOIN orders_break_down 
    ON orders.order_id = orders_break_down.order_id
    WHERE orders.country = 'France'
    GROUP BY orders.country, orders_break_down.category
    ORDER BY avg_sales DESC 

'''

customers_country = sql.read_sql(sql_countries, con = db_connection)
customers_country.head()

,country,category,avg_sales
0,France,Furniture,2632.707860
1,France,Technology,2508.548575
2,France,Office Supplies,791.520390


#### 13. Find the country, category and average sales where average sales is the highest

In [267]:
# Find the country, category and average sales (i.e. avg( unit_price * quantity * (1 - discount) ) ) where average sales is the highest
sql_countries = '''
    SELECT 
        orders.country,
        orders_break_down.category,
        AVG(unit_price * quantity * (1 - discount)) AS avg_sales
    FROM orders
    JOIN orders_break_down 
    ON orders.order_id = orders_break_down.order_id
    GROUP BY orders.country, orders_break_down.category
    ORDER BY avg_sales DESC LIMIT 1

'''

customers_country = sql.read_sql(sql_countries, con = db_connection)
customers_country.head()

,country,category,avg_sales
0,Switzerland,Technology,6487.846154


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams.
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  



---



---



> > > > > > > > > © 2025 Institute of Data


---



---



